In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# Function to read data from a file
def read_data(file_location):
    df = pd.read_csv(file_location)
    return df

In [ ]:
def df_one_day(pandas_df_to_plot,year_to_plot,month_to_plot, day_to_plot):
    p_one_day = pandas_df_to_plot[(pandas_df_to_plot.index.day == day_to_plot) & (pandas_df_to_plot.index.year == year_to_plot) & (pandas_df_to_plot.index.month == month_to_plot)]
    return p_one_day

In [ ]:
def df_one_month(pandas_df_to_plot,year_to_plot,month_to_plot, day_to_plot):
    p_one_day = pandas_df_to_plot[(pandas_df_to_plot.index.day == day_to_plot) & (pandas_df_to_plot.index.year == year_to_plot) & (pandas_df_to_plot.index.month == month_to_plot)]
    return p_one_day

In [ ]:
def plot_one_day(pandas_df_to_plot,year_to_plot,month_to_plot, day_to_plot):
    p_to_plot = df_one_day(pandas_df_to_plot,year_to_plot,month_to_plot, day_to_plot)
    plt.rcParams["figure.figsize"] = (40,3)
    plt.plot(p_to_plot)
    plt.show()
    print("Plotting: year = ", year_to_plot, " month = ", month_to_plot, " day = ", day_to_plot)

In [ ]:
def plot_one_month(pandas_df_to_plot,year_to_plot,month_to_plot):
    days_from_month = pandas_df_to_plot[(pandas_df_to_plot.index.year == year_to_plot) & (pandas_df_to_plot.index.month == month_to_plot)].index.day.unique().values
    for day in days_from_month:
        plot_one_day(pandas_df_to_plot, year_to_plot,month_to_plot, day)

In [ ]:
def print_max_freq_month(df):
    door_open_monthly = df.resample('M').sum()
    max_month_year = door_open_monthly[door_open_monthly.value == door_open_monthly.value.max()].index.year[0]
    max_month_month = door_open_monthly[door_open_monthly.value == door_open_monthly.value.max()].index.month[0]
    print('Year with max door opening:', max_month_year)
    print('Month with max door opening:', max_month_month)
    plot_one_month(df,max_month_year,max_month_month)

In [ ]:
def get_all_months(df, year):
    return df[(df.index.year == year)].index.month.unique().values

In [ ]:
def get_all_years(df):
    return df.index.year.unique().values

In [ ]:
def plot_one_year(df, year):
    all_months = get_all_months(df, year)
    for month in all_months:
        plot_one_month(df, year, month)

In [ ]:
def show_all_years(df):
    all_years = get_all_years(df)
    for year in all_years:
        plot_one_year(df, year)

In [ ]:
# Function to preprocess and clean data
def preprocess_data(df, aggregation='H', ws=24, number_of_predicted_days=2):

    df.dropna(inplace=True)

    df.drop_duplicates(subset=['source_ts'], inplace=True)

    datetime_series = pd.to_datetime(df['source_ts'])
    datetime_index = pd.DatetimeIndex(datetime_series.values)
    df=df.set_index(datetime_index)
    df.drop('source_ts',axis=1,inplace=True)

    df=df.asfreq(freq='S', method='ffill')

    lastDay = df.index[-1].strftime('%Y-%m-%d')
    df = df.loc[:lastDay].iloc[:-1 , :]
    df

    prediction_in_future_time = ws * number_of_predicted_days
    
    df_resampled = df.resample(aggregation).sum()
    df_resampled
    
    year_to_plot = df.index[-1].year
    month_to_plot = df.index[-1].month
    day_to_plot = df.index[-1].day

    plot_one_day(df, year_to_plot, month_to_plot, day_to_plot)

    plot_one_day(df_resampled, year_to_plot, month_to_plot, day_to_plot)

    df = df_resampled
    n_splits = 4
    test_size = 48
    total_len = len ( df )
    fold_size = (total_len - test_size) // n_splits
    
    tscv = TimeSeriesSplit ( n_splits = n_splits)
    splits = []
    for train_index, test_index in tscv.split(df):
        test_indices = np.arange(test_index[0], test_index[0] + test_size)
        train_indices = np.arange(0, test_indices[0])
        splits.append((train_indices[0], train_indices[-1], test_indices[0], test_indices[-1]))

    return df_resampled, splits, n_splits

In [ ]:
# Function to create an LSTM model
def create_rnn_model(shape):
    Model_P = Sequential()
    Model_P.add(LSTM(units = 64,  return_sequences = True, input_shape = (shape, 1)))
    Model_P.add(Dropout(0.2))
    Model_P.add(LSTM(units=64, return_sequences=True)) 
    Model_P.add(Dropout(0.2)) 
    Model_P.add(LSTM(units=64, return_sequences=True))
    Model_P.add(Dropout(0.2))         
    Model_P.add(LSTM(units=64))
    Model_P.add(Dropout(0.2)) 
    Model_P.add(Dense(units=1))
    return Model_P

In [ ]:
# Function to train the LSTM model
def train_lstm_model(model, x_train, y_train, epochs=15, batch_size=32):
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

In [ ]:
# Function to make predictions using the trained model
def make_predictions(model, batch_one, prediction_in_future_time):
    prediction_test = []
    batch_new = batch_one.reshape((1, ws, 1))
    
    for _ in range(prediction_in_future_time):
        first_pred = model.predict(batch_new)[0]
        prediction_test.append(first_pred)
        batch_new = np.append(batch_new[:, 1:, :], [[first_pred]], axis=1)

    return np.array(prediction_test)


In [ ]:
# Function to evaluate the model
def evaluate_model(test_set, predictions):
    rmse = np.sqrt(mean_squared_error(test_set, predictions))
    rsquare = r2_score(test_set, predictions)
    return rmse, rsquare

In [ ]:
# Function to compute the forecast bias
def compute_forecast_bias(test_set, predictions):
    differences = [predicted - actual for predicted, actual in zip(predictions, test_set)]
    bias = sum(differences) / len(differences)
    return bias

In [ ]:
# Function to compute the forecast interval coverage
def compute_forecast_interval_coverage(test_set, predictions, rmse):
    forecasted_intervals = [(lower, upper) for lower, upper in zip(predictions - 1.96 * rmse, predictions + 1.96 * rmse)]
    num_within_the_interval = sum((lower <= actual <= upper) for actual, (lower, upper) in zip (test_set, forecasted_intervals))
    total_observations = len(test_set)
    fic = num_within_the_interval / total_observations * 100
    return fic

In [ ]:
def compute_prediction_direction_accuracy(test_set, predictions):
    correct_directions = 0
    for i in range(1, len(predictions)):
        pred_change = predictions[i] - predictions[i-1]
        actual_change = test_set[i] - test_set[i-1]
        if(pred_change > 0 and actual_change > 0) or (pred_change > 0 and actual_change < 0) or (pred_change == 0 and actual_change == 0):
            correct_directions += 1
    pda = (correct_directions / (len(predictions) - 1)) * 100
    return pda

In [ ]:
# # Function to plot the actual vs predicted values
def plot_results(test_set, predictions):
    plt.rcParams["figure.figsize"] = (40,3)
    plt.plot(test_set, color='green', label='Actual value')
    plt.plot(predictions, color='orange', label='Predicted value')
    plt.legend()
    plt.show()

In [ ]:
# Function to create input sequences for training the LSTM model
def create_input_sequences(data, ws):
    x_train, y_train = [], []

    for i in range(ws, len(data)):
        x_train.append(data[i-ws:i, 0:1])
        y_train.append(data[i, 0])
    
    return np.array(x_train), np.array(y_train)

In [ ]:
# Main function
def main(file_location):
    df = read_data(file_location)

    df_resampled, splits, n_splits = preprocess_data(df)

    rmse_values = []
    r_squared_values = []
    for split in range(n_splits):
        train_start, train_end, test_start, test_end = splits[split]
        training_set = df_resampled[train_start:train_end + 1].values
        test_set = df_resampled[test_start:test_end + 1].values
        sc = MinMaxScaler(feature_range=(0,1))
        training_set_scaled = sc.fit_transform(training_set)
        x_train, y_train = create_input_sequences(training_set_scaled, ws)
        model = create_rnn_model(x_train.shape[1])
        model = train_lstm_model(model, x_train, y_train)
        batch_one = training_set_scaled[-ws:]
        prediction_in_future_time = test_end - test_start + 1
        prediction = make_predictions(model, batch_one, prediction_in_future_time)
        prediction = sc.inverse_transform(prediction)
        rmse, rsquare = evaluate_model(test_set, prediction)
        rmse_values.append(rmse)
        r_squared_values.append(rsquare)
        plot_results(test_set, prediction)

    print("RMSE values= ",rmse_values)
    print("R2 values= ", r_squared_values)

    print("Average RMSE: ", np.mean(rmse_values))
    print("Average R2: ", np.mean(r_squared_values))

In [ ]:
ws = 24
prediction_in_future_time = ws * 2
main("./testData/Dataset1.csv")